In [ ]:
# Future improvements
# Separate dir for each step
# Create JSON array to assign IDs, 
#     keep track of PDF files process (each step?) etc.
# Remove numbers, urls
# Change variable names of jsonDoc, jsonInv, jsonGram to avoid redefiining argument

In [1]:
# Imports
import os # For file/directory interaction
import time, sys
from datetime import datetime, date # For log data
import re # For text replacement
import spacy # Pipeline processes (stopword and punctuation removal, lemmatization)
from nltk.stem.snowball import SnowballStemmer # Pipeline process for stemming
import json
nlp = spacy.load("en_core_web_sm")
txtFilesDir = 'Text Files'
rtnFilesDir = 'n Removed'
spaceFilesDir = 'No Spaces'
swFilesDir = 'Stop Words'
engFilesDir = 'English Words'
stemFilesDir = 'Stemmed'
jsonDocIndex = 'doc_dictionary.json'
jsonInvIndex = 'inverted_index.json'
jsonGramIndex = 'gram_index.json'

absolute = 'C:/Users/micah/Documents/IWU/CIS Practicum/Files'

In [3]:
# Pre-condition: All PDF files to be processed are in the sub-directory
#     pdfDir, and pdfDir is in absPath. absPath is by default the 
#     directory in which the program is executed
# Post-condition: All PDF files processed without error are converted to
#     text files which are placed in a new sub-directory 'Text Files'
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter, process_pdf#process_pdf
from pdfminer.pdfparser import PDFParser
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams

from io import StringIO

def getText(pdfPath):
    rsrcmgr = PDFResourceManager()
    sio = StringIO()
    laparams = LAParams()
    device = TextConverter(rsrcmgr, sio, laparams=laparams)
#     interpreter = PDFPageInterpreter(rsrcmgr, device)
    
    
    with open(pdfPath, 'rb') as pdfFile:
        process_pdf(rsrcmgr, device, pdfFile)
#         parser = PDFParser(pdfFile)
#         doc.set_parser(parser)
#         interpreter.process_pdf(doc)
#             layout = device.get_result()
#             for element in layout:
#                 if instanceof(element, LTTextBoxHorizontal):
#                     print(element.get_text)
                
    
    text = sio.getvalue()
    
    device.close()
    sio.close()
    
    return text

def pdfToText(pdfDir, absPath = os.getcwd(), txtDir = txtFilesDir):
    
    pdfPath = absPath+'/'+pdfDir
    txtPath = absPath+'/'+txtDir
    if pdfDir not in os.listdir(absPath):
        print('The specified directory "' + directory + '" does not exist')
        return
# Creates 'Text Files' directory for converted PDFs
    if txtDir not in os.listdir(absPath):
        os.mkdir(txtPath)
    
    docNum = 0
    stopAt = 25
    totalNum = len([file for file in os.scandir(pdfPath) if file.name.endswith('.pdf')])
    with open(absPath+'/'+'log.txt', 'a+', encoding="utf-8") as log:
        log.write("PDF to Text\n" + date.today().strftime("%m/%d/%y") +
                  " at " + datetime.now().strftime("%H:%M:%S") + "\n\n")        
# Moves on to next entity if the current entity is not a PDF
        for entity in os.scandir(pdfPath):
            if not entity.name.endswith('.pdf'):
                continue
            docNum += 1
            index = -4 # Remove '.pdf' from file name when creating '.txt' file
            fileName = entity.name[:index]+'.txt'
            print("Now on '"+entity.name+"'. . . ", end='')
            
# This block attempts to read the PDF file, extract text from each page,
#     and write the text to a text file with the same name
# Some documents are protected, corrupted, etc. and text cannot be extracted
# Exceptions are recorded in log.txt
# hasError remains true until each step in the try block is complete
            if fileName not in os.listdir(txtPath): 
                hasError = True
                try:
                    text = getText(pdfPath+'/'+entity.name)
                    txtFile = open(txtPath+'/'+fileName, 'w+', encoding="utf-8")
                    txtFile.write(text)
                    print("done\n")
                    hasError = False
#                     except TypeError as e:
#                         log.write("TypeError: " + entity.name + ":\n" + str(e))
#                     except PyPDF2.utils.PdfReadError as e:
#                         log.write("PdfReadError: " + entity.name + ":\n" + str(e))
#                     except OSError as e:
#                         log.write("OSError: " + entity.name + ":\n" + str(e))
#                     except decimal.InvalidOperation as e:
#                         log.write("InvalidOperation: " + entity.name + ":\n" + str(e))
                except Exception as e:
                    log.write(str(docNum)+": " + entity.name + ": \n\t" + str(e)+"\n")

                if hasError:
                    print("there was an error reading this document. See log for details. Reference number "+str(docNum)+".\n")
            else:
                max = 10
                if len(fileName) > max:
                    print('"'+fileName[:max]+'..."', end='')
                else:
                    print('"'+fileName+'"', end='')
                print(' already exists\n')
            if docNum == stopAt:
                print("PDF to Text was stopped after "+str(docNum)+" documents.")
                break
        log.write("\n\n")
pdfToText('Working', absolute)

Now on 'Lafferty_pcfg-notes.pdf'. . . "Lafferty_p..." already exists

Now on 'LearningExecutableSemanticParsers.pdf'. . . "LearningEx..." already exists

Now on 'p123-zhang.pdf'. . . "p123-zhang..." already exists

Now on 'Partial Parsing Finite-State Cascades.pdf'. . . "Partial Pa..." already exists

Now on 'QueryEffectiveness.pdf'. . . "QueryEffec..." already exists

Now on 'Text Clustering Algorithms.pdf'. . . "Text Clust..." already exists

Now on 'Text-Analytics-and-Natural-Language-Processing--.pdf'. . . "Text-Analy..." already exists

Now on 'Workshop on Robust Methods in Analysis of Natural Language Data.pdf'. . . "Workshop o..." already exists



In [ ]:
# Work on 10 (good) files at a time until pipeline works
#   then incrementally add files and clean up errors

# Function to remove \n
def rmvN(txtDir = txtFilesDir, rtnDir = rtnFilesDir, absPath = os.getcwd()):
    txtPath = absPath+'/'+txtDir
    if txtDir not in os.listdir(absPath):
        print('The specified directory "' + txtPath + '" does not exist')
        return
    rtnPath = absPath+'/'+rtnDir
    if rtnDir not in os.listdir(absPath):
        os.mkdir(rtnPath)
        
    for entity in os.scandir(txtPath):
        print("Now on '"+entity.name+"'. . . ", end='')
        with open(txtPath+'/'+entity.name, 'r+', encoding='utf-8') as txtFile:
            with open(rtnPath+'/'+entity.name, 'w+', encoding='utf-8') as rtnFile:
                text = txtFile.read()
                text = re.sub('-\n', '', text)
                text = re.sub('\n', '', text)
                rtnFile.write(text)
                rtnFile.truncate()
        print("done")
rmvN(absPath = absolute)

In [ ]:
# Funtion to move files without spaces to new 'Without Spaces' directory         
def checkSpaces(txtDir = rtnFilesDir, spacesDir = spaceFilesDir, absPath = os.getcwd()):
    txtPath = absPath+'/'+txtDir
    if txtDir not in os.listdir(absPath):
        print('The specified directory "' + txtPath + '" does not exist')
        return
    spacesPath = absPath+'/'+spacesDir
    if spacesDir not in os.listdir(absPath):
        os.mkdir(spacesPath)
        
    with open(absPath+'/'+'Spaces.txt', 'a+', encoding='utf-8') as spaces: 
        spaces.write("Check Spaces\n" + date.today().strftime("%m/%d/%y") +
                  " at " + datetime.now().strftime("%H:%M:%S") + "\n\n")
        for entity in os.scandir(txtPath):
            print("Now on '"+entity.name+"'. . . ", end='')
            txtFile = open(txtPath+'/'+entity.name, 'r', encoding='utf-8')
            text = txtFile.read()
            split = text.split(' ')
            if len(split) < len(text)/10 or len(text) < 100 or text == '':
                txtFile.close()
                spaces.write(entity.name+'\n')
                if entity.name not in os.listdir(spacesPath):
                    os.rename(txtPath+'/'+entity.name, spacesPath+'/'+entity.name)
                else:
                    os.remove(txtPath+'/'+entity.name)
            print("done")
        spaces.write('\n\n')
checkSpaces(absPath = absolute) 

In [ ]:
# Function to remove stopwords
# NLTK or SpaCy
# Inverted File: gram:[doc1, doc3] or gram:[[doc1,freq], [doc3,freq]]
def rmvStopWords(nlp, txtDir = rtnFilesDir, swDir = swFilesDir, absPath = os.getcwd()):
    txtPath = absPath+'/'+txtDir
    if txtDir not in os.listdir(absPath):
        print('The specified directory "' + txtPath + '" does not exist')
        return
    swPath = absPath+'/'+swDir
    if swDir not in os.listdir(absPath):
        os.mkdir(swPath)

    for entity in os.scandir(txtPath):
        print("Now on '"+entity.name+"'. . . ", end='')
        with open(txtPath+'/'+entity.name, 'r+', encoding='utf-8') as txtFile:
            with open(swPath+'/'+entity.name, 'w+', encoding='utf-8') as swFile:
                doc = nlp(txtFile.read())
                noStopWords = [token.lemma_.lower() for token in doc if not token.is_stop and not token.is_punct and not token.text.isnumeric()]
                swFile.write(" ".join(noStopWords))
                swFile.truncate()
        print("done")

rmvStopWords(nlp, absPath = absolute)

In [ ]:
# Remove non-english words
def rmvNonEng(txtDir = swFilesDir, engDir = engFilesDir, absPath = os.getcwd()):
    txtPath = absPath+'/'+txtDir
    if txtDir not in os.listdir(absPath):
        print('The specified directory "' + txtPath + '" does not exist')
        return
    engPath = absPath+'/'+engDir
    if engDir not in os.listdir(absPath):
        os.mkdir(engPath)
    with open(absPath+'/'+'words_dictionary.json') as json_file:
        words = json.load(json_file)
        
    lets = []
    alph = 'abcdefghijklmnopqrstuvwxyz'
    for let in alph:
        lets.append(let)
        for char in alph:
            lets.append(let+char)
        
    for entity in os.scandir(txtPath):
        print("Now on '"+entity.name+"'. . . ", end='')
        with open(txtPath+'/'+entity.name, 'r+', encoding='utf-8') as txtFile:
            with open(engPath+'/'+entity.name, 'w+', encoding='utf-8') as engFile:
                text = txtFile.read().split(' ')
                engChars = [word for word in text if word in words and word not in lets]
                engFile.write(" ".join(engChars))
                engFile.truncate()
        print("done")
rmvNonEng(absPath = absolute)

In [ ]:
# Stem words in all documents
def stem(txtDir = engFilesDir, stemDir = stemFilesDir, absPath = os.getcwd()):
    txtPath = absPath+'/'+txtDir
    if txtDir not in os.listdir(absPath):
        print('The specified directory "' + txtPath + '" does not exist')
        return
    stemPath = absPath+'/'+stemDir
    if stemDir not in os.listdir(absPath):
        os.mkdir(stemPath)
        
    stemmer = SnowballStemmer(language='english')
    for entity in os.scandir(txtPath):
        print("Now on '"+entity.name+"'. . . ", end='')
        with open(txtPath+'/'+entity.name, 'r+', encoding='utf-8') as txtFile:
            with open(stemPath+'/'+entity.name, 'w+', encoding='utf-8') as stemFile:
                text = txtFile.read().split(' ')
                stemmed = [stemmer.stem(word) for word in text]
                stemFile.write(" ".join(stemmed))
                stemFile.truncate()
        print("done")
stem(absPath = absolute)

In [4]:
# Create JSON with file_name : doc_id
def update_doc_index(jsonDoc = jsonDocIndex, txtDir = stemFilesDir, absPath = os.getcwd()):
    txtPath = absPath+'/'+txtDir
    jsonPath = absPath+'/'+jsonDoc
    if jsonDoc in os.listdir(absPath):
        with open(jsonPath, 'r') as jsonFile:
            docIndex = json.load(jsonFile)
    else:
        docIndex = {}
    for fileName in os.listdir(txtPath):
        if fileName not in docIndex.values():
            docIndex[len(docIndex)+1] = fileName
    with open(jsonPath, 'w') as jsonFile:
        json.dump(docIndex, jsonFile, indent=4)

update_doc_index(absPath = absolute)

In [5]:
# Create JSON with {doc_id1 : {"gram1":freq, "gram2":freq}, doc_id2 : {"gram1":freq}}
def update_inv_index(ngrams, jsonDoc = jsonDocIndex, jsonInv = jsonInvIndex, txtDir = stemFilesDir, absPath = os.getcwd()):
    txtPath = absPath+'/'+txtDir
    jsonDocPath = absPath+'/'+jsonDoc
    if txtDir not in os.listdir(absPath):
        print('The specified directory "' + txtPath + '" does not exist')
        return
    if jsonDoc not in os.listdir(absPath):
        print('FILE NOT FOUND: The specified file "' + jsonDocPath + '" does not exist')
        return
    jsonInvPath = absPath+'/'+jsonInv
    if jsonInv in os.listdir(absPath):
        with open(jsonInvPath, 'r') as jsonFile:
            invIndex = json.load(jsonFile)
    else:
        invIndex = {}
#     Loads document index
    with open(jsonDocPath, 'r') as jsonFile:
        docIndex = json.load(jsonFile)
        
    for ngram in ngrams:
        for docID in docIndex:
            if docID not in invIndex:
                invIndex[docID] = {}
            if ngram not in invIndex[docID]:
                invIndex[docID][ngram] = {}
                with open(txtPath+'/'+docIndex[docID], 'r', encoding='utf-8') as txtFile:
                    text = txtFile.read().split(' ')
                    while len(text) > ngram-1:
                        term = " ".join(text[:ngram])
                        if term in invIndex[docID][ngram]:
                            invIndex[docID][ngram][term] += 1
                        else:
                            invIndex[docID][ngram][term] = 1
                        text.pop(0)
    with open(jsonInvPath, 'w') as jsonFile:
        json.dump(invIndex, jsonFile, indent=4)

update_inv_index(list(range(2,4)), absPath = absolute)

In [6]:
# Create JSON with {doc_id1 : {"gram1":freq, "gram2":freq}, doc_id2 : {"gram1":freq}}
def update_gram_index(minFreq = 1, jsonInv = jsonInvIndex, jsonGram = jsonGramIndex, txtDir = stemFilesDir, absPath = os.getcwd()):
#     Makes sure all files and directories exist (jsonInv, txtDir)
    jsonInvPath = absPath+'/'+jsonInv
    if jsonInv not in os.listdir(absPath):
        print('FILE NOT FOUND: The specified file "' + jsonInvPath + '" does not exist')
        return
#     jsonGram is created if it does not exist
    jsonGramPath = absPath+'/'+jsonGram
    if jsonGram in os.listdir(absPath):
        with open(jsonGramPath, 'r') as jsonFile:
            gramIndex = json.load(jsonFile)
    else:
        gramIndex = {}
#     Loads Inverted File index
    with open(jsonInvPath, 'r') as jsonFile:
        invIndex = json.load(jsonFile)
        
    for docID, ngrams in invIndex.items():
        for terms in ngrams.values():
            for term, freq in terms.items():
                if freq >= minFreq:
                    if term not in gramIndex:
                        gramIndex[term] = {}
                    gramIndex[term][docID] = freq
#     Writes gramIndex to gram JSON file
    with open(jsonGramPath, 'w') as jsonFile:
        json.dump(gramIndex, jsonFile, indent=4)

update_gram_index(absPath = absolute)              

In [7]:
# Print grams found in 3 or more documents
def getGrams(numDocs = 3, jsonDoc = jsonDocIndex, jsonGram = jsonGramIndex, txtDir = stemFilesDir, absPath = os.getcwd()):
#     Makes sure all files and directories exist (textDir, jsonDoc, txtDir)
    txtPath = absPath+'/'+txtDir
    if txtDir not in os.listdir(absPath):
        print('The specified directory "' + txtPath + '" does not exist')
        return
    jsonDocPath = absPath+'/'+jsonDoc
    if jsonDoc not in os.listdir(absPath):
        print('FILE NOT FOUND: The specified file "' + jsonDocPath + '" does not exist')
        return
    jsonGramPath = absPath+'/'+jsonGram
    if jsonGram not in os.listdir(absPath):
        print('FILE NOT FOUND: The specified file "' + jsonGramPath + '" does not exist')
        return
#    Loads document index
    with open(jsonDocPath, 'r') as jsonFile:
        docIndex = json.load(jsonFile)
#    Loads gram index 
    with open(jsonGramPath, 'r') as jsonFile:
        gramIndex = json.load(jsonFile)
    
    for gram, docs in gramIndex.items():
        if len(docs) >= numDocs:
            print("\""+gram+"\" found in "+str(len(docs))+" documents")
            for docID in docs:
                print("\t"+ str(docIndex[docID]))
                
getGrams(5, absPath = absolute)

"natur languag process" found in 5 documents
	LearningExecutableSemanticParsers.txt
	Text-Analytics-and-Natural-Language-Processing--.txt
	Workshop on Robust Methods in Analysis of Natural Language Data.txt
	p123-zhang.txt
	QueryEffectiveness.txt
"natur languag" found in 8 documents
	Lafferty_pcfg-notes.txt
	LearningExecutableSemanticParsers.txt
	p123-zhang.txt
	Partial Parsing Finite-State Cascades.txt
	QueryEffectiveness.txt
	Text Clustering Algorithms.txt
	Text-Analytics-and-Natural-Language-Processing--.txt
	Workshop on Robust Methods in Analysis of Natural Language Data.txt
"comput scienc" found in 6 documents
	Lafferty_pcfg-notes.txt
	LearningExecutableSemanticParsers.txt
	QueryEffectiveness.txt
	Text Clustering Algorithms.txt
	Text-Analytics-and-Natural-Language-Processing--.txt
	Workshop on Robust Methods in Analysis of Natural Language Data.txt
"languag process" found in 5 documents
	LearningExecutableSemanticParsers.txt
	p123-zhang.txt
	QueryEffectiveness.txt
	Text-Analytics-

In [22]:
import math
def calcCosSim(list1, list2):
    if len(list1) != len(list2):
        print("Lists do not have the same number of elements")
        return -1
    sumProd = sum(n1*n2 for n1, n2 in zip(list1, list2))
    sumL1 = sum(n**2 for n in list1)
    sumL2 = sum(n**2 for n in list2)
    
    return sumProd/(math.sqrt(sumL1)*math.sqrt(sumL2))

# Create vectors for all n-grams (with freq>1?) between two docs, multiply them,
#     then divide by the product Euclidian norms
# Print out similar phrases
def cossim(ngrams, jsonDoc = jsonDocIndex, jsonInv = jsonInvIndex, absPath = os.getcwd()):
    if jsonDoc not in os.listdir(absPath):
        print('FILE NOT FOUND: The specified file "' + txtPath + '" does not exist')
        return
    jsonDocPath = absPath+'/'+jsonDoc
    if jsonInv not in os.listdir(absPath):
        print('FILE NOT FOUND: The specified file "' + txtPath + '" does not exist')
        return
    
    jsonInvPath = absPath+'/'+jsonInv
    with open(jsonDocPath, 'r') as jsonDoc:
        docIndex = json.load(jsonDoc)
    with open(jsonInvPath, 'r') as jsonInv:
        invIndex = json.load(jsonInv)
            
        
    for doc1 in invIndex:
        doc1Terms = []
        for ngram, terms in invIndex[doc1].items():
            if int(ngram) in ngrams:
                for term in terms:
                    doc1Terms.append(term)
        for doc2 in invIndex:
            if doc2 > doc1:
                print(docIndex[doc1]+" and "+docIndex[doc2])
                allTerms = doc1Terms.copy()
                likeTerms = []
                for ngram, terms in invIndex[doc2].items():
                    if int(ngram) in ngrams:
                        for term in terms:
                            if term not in allTerms:
                                allTerms.append(term)
                            else:
                                likeTerms.append(term)
                allTerms.sort()
                doc1freq = []
                doc2freq = []
                for term in allTerms:
                    weight = ngrams[len(term.split())]
                    ngram = str(len(term.split()))
                    doc1freq.append(invIndex[doc1][ngram].get(term, 0)*weight)
                    doc2freq.append(invIndex[doc2][ngram].get(term, 0)*weight)
                cosSimilarity = calcCosSim(doc1freq, doc2freq)
                print("Like terms (doc1, doc2):")
                for term in likeTerms:
                    print("\t"+term+" ("+str(invIndex[doc1][str(len(term.split()))][term])+", "+str(invIndex[doc2][str(len(term.split()))][term])+")")
                numLike = len(likeTerms)
                print("Doc1 has "+str(len(doc1Terms)-numLike)+" unique terms")
                print("Doc2 has "+str(len(allTerms)-len(doc1Terms))+" unique terms")
                print("Total like terms: \t"+str(numLike))
                print("Total unlike terms: \t"+str(len(allTerms)-numLike))
                print("Cosine Similarity: "+ str(cosSimilarity), end='\n\n')
gramWeight = {2:1, 3:5}
cossim(gramWeight, absPath = absolute)
    

Lafferty_pcfg-notes.txt and LearningExecutableSemanticParsers.txt
Like terms (doc1, doc2):
	natur languag (1, 48)
	grammar model (1, 2)
	actual comput (1, 1)
	denot set (1, 3)
	speech recognit (4, 1)
	statist model (1, 3)
	syntact pars (1, 1)
	consist follow (1, 1)
	probabl deriv (1, 3)
	rule form (2, 2)
	deriv general (1, 1)
	context free (9, 1)
	free grammar (4, 1)
	rule build (1, 1)
	languag model (5, 1)
	number time (5, 2)
	assign probabl (4, 3)
	probabl give (2, 1)
	rule grammar (1, 1)
	paramet respect (1, 1)
	paramet deriv (1, 1)
	maxim likelihood (2, 1)
	likelihood train (2, 1)
	train datum (1, 2)
	log likelihood (3, 2)
	simpl approach (1, 1)
	updat paramet (2, 1)
	languag grammar (3, 2)
	form basic (1, 1)
	point view (1, 2)
	valid sentenc (1, 1)
	model deriv (1, 1)
	featur base (1, 2)
	pars algorithm (1, 1)
	pars chart (1, 1)
	estim model (1, 1)
	lexic rule (1, 3)
	paramet estim (1, 1)
	introduc new (1, 1)
	give new (1, 1)
	probabilist grammar (1, 1)
	comput scienc (1, 1)
	cont

Like terms (doc1, doc2):
	natur languag (48, 23)
	artifici intellig (5, 5)
	languag process (9, 17)
	model uncertainti (1, 2)
	machin learn (6, 10)
	word appear (1, 1)
	hard code (1, 2)
	model build (1, 1)
	challeng scale (1, 1)
	level accuraci (1, 1)
	system train (1, 1)
	train datum (2, 1)
	datum resourc (1, 1)
	appropri action (1, 1)
	comput scienc (1, 1)
	comput linguist (13, 1)
	human languag (1, 1)
	unstructur text (1, 6)
	task machin (1, 1)
	larg collect (1, 1)
	model model (2, 1)
	allow improv (1, 1)
	neural network (2, 1)
	result semant (1, 1)
	deep understand (3, 1)
	result base (1, 2)
	exampl includ (1, 1)
	name entiti (2, 1)
	simpl approach (1, 1)
	interfac natur (1, 1)
	natur languag process (9, 17)
	interfac natur languag (1, 1)
Doc1 has 5347 unique terms
Doc2 has 4434 unique terms
Total like terms: 	32
Total unlike terms: 	9781
Cosine Similarity: 0.05353784772126628

LearningExecutableSemanticParsers.txt and Workshop on Robust Methods in Analysis of Natural Language Data

Like terms (doc1, doc2):
	social network (5, 8)
	text measur (1, 1)
	use text (1, 1)
	topic model (1, 26)
	discuss text (1, 2)
	network base (1, 3)
	follow section (1, 1)
	describ section (1, 1)
	section describ (2, 1)
	base network (1, 1)
	section discuss (1, 3)
	case term (1, 2)
	document text (1, 3)
	previous case (2, 1)
	use semant (1, 1)
	inform retriev (2, 12)
	case vector (1, 1)
	document corpus (1, 6)
	function measur (1, 2)
	weight term (1, 4)
	term vector (1, 1)
	similar measur (1, 2)
	search engin (2, 1)
	measur similar (1, 2)
	search process (1, 1)
	high similar (1, 1)
	case document (4, 4)
	run time (2, 5)
	document contain (1, 5)
	exampl method (1, 1)
	particular applic (1, 1)
	case start (1, 1)
	document choos (1, 1)
	discuss use (1, 4)
	natur languag (1, 2)
	term occur (1, 2)
	point time (1, 1)
	work model (1, 1)
	similar term (1, 1)
	node link (1, 1)
	kind link (1, 1)
	link base (1, 3)
	text associ (1, 1)
	multi dimension (3, 1)
	link propos (1, 1)
	form natur (1, 1)
	

Like terms (doc1, doc2):
	studi datum (2, 1)
	improv retriev (3, 1)
	reduct topic (1, 1)
	contain descript (1, 1)
	word word (1, 3)
	data set (1, 6)
	set construct (1, 1)
	inform retriev (14, 12)
	observ suggest (1, 1)
	hypothesi hypothesi (2, 1)
	topic topic (1, 1)
	word second (1, 1)
	word associ (1, 1)
	natur languag (2, 2)
	associ document (1, 1)
	goal topic (1, 1)
	topic size (2, 1)
	task perform (2, 2)
	time depend (1, 1)
	correl analysi (2, 1)
	generat topic (1, 1)
	experiment result (1, 2)
	intern confer (3, 1)
	retriev document (2, 1)
	comput scienc (2, 1)
	text retriev (3, 1)
	univers ithaca (2, 1)
Doc1 has 1824 unique terms
Doc2 has 13328 unique terms
Total like terms: 	27
Total unlike terms: 	15152
Cosine Similarity: 0.002036330979646199

QueryEffectiveness.txt and Text-Analytics-and-Natural-Language-Processing--.txt
Like terms (doc1, doc2):
	natur languag (2, 23)
	languag process (1, 17)
	search result (1, 4)
	effect primari (1, 1)
	comput scienc (2, 1)
	user interfac (1, 

Like terms (doc1, doc2):
	natur languag (23, 45)
	comput linguist (1, 26)
	exact know (2, 1)
	artifici intellig (5, 1)
	human comput (1, 1)
	languag process (17, 20)
	difficult problem (1, 1)
	includ content (1, 1)
	help user (1, 1)
	probabl predict (1, 4)
	measur model (1, 1)
	model predict (6, 2)
	abl predict (1, 2)
	model show (1, 2)
	human languag (1, 1)
	generat model (2, 4)
	text datum (3, 1)
	high level (2, 8)
	pattern match (1, 3)
	improv perform (2, 1)
	inform decis (1, 1)
	show number (1, 2)
	comput scienc (1, 1)
	process natur (1, 1)
	languag write (1, 1)
	datum need (1, 1)
	datum good (1, 1)
	process extract (1, 1)
	rate data (1, 3)
	produc datum (1, 2)
	type inform (1, 2)
	datum drive (1, 1)
	effect nois (1, 1)
	linguist rule (2, 1)
	approach implement (1, 1)
	year ago (3, 1)
	relev result (3, 1)
	automat translat (2, 2)
	larg collect (1, 2)
	amount datum (2, 2)
	word appear (1, 2)
	improv qualiti (1, 1)
	datum mean (2, 1)
	start point (1, 2)
	provid inform (1, 1)
	extract

In [ ]:
# Trigram parser-based inverted file 
# (TF-DIF to remove trigrams common to most or all documents)

In [ ]:
# Clustering algorithm based on trigram inverted file

In [ ]:
# Add bigram parser-based info to inverted file

In [ ]:
# Implement clustering on bigram inverted file